In [1]:
import numpy as np
from tqdm import tqdm

import os
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

# Load Dataset
## The dataset may be 'cora', 'citeseer', 'pubmed'

In [2]:
from graphgallery.datasets import Planetoid
data = Planetoid('cora', root="~/GraphData/datasets/", verbose=False)
graph = data.graph
splits = data.split_nodes()

# Init

In [3]:
table = []
device = "gpu"
runs = 10

GNNs = ("GCN", "SGC", "GAT")
Epochs = (200, 100, 200)
Backends = ("TensorFlow", "PyTorch", "PyG", "DGL")

# Benchmarking Experiments

In [4]:
from graphgallery import gallery
from graphgallery import set_backend

for backend in Backends:
    set_backend(backend)
    tmp = []
    for Net, epochs in zip(GNNs, Epochs):
        reports = []
        for seed in range(runs):
            trainer = getattr(gallery, Net)(graph, device=device, seed=42+seed).process(attr_transform="normalize_attr").build()
            trainer.fit(splits.train_nodes, splits.val_nodes, verbose=0, epochs=epochs)
            results = trainer.evaluate(splits.test_nodes, verbose=0) 
            reports.append(results.accuracy)

        reports = f"{np.mean(reports):.2%}±{np.std(reports):.2%}"
        print(f"{trainer}: \033[31m{reports}\033[0m")        
        tmp.append(reports)
    table.append(tmp)


GCN(device=GPU:0, backend=TensorFlow 2.1.2 Backend): 80.48%±0.97%
SGC(device=GPU:0, backend=TensorFlow 2.1.2 Backend): 81.33%±0.61%
GAT(device=GPU:0, backend=TensorFlow 2.1.2 Backend): 81.96%±1.02%
GCN(device=cuda:0, backend=PyTorch 1.6.0+cu101 Backend): 81.25%±0.88%
SGC(device=cuda:0, backend=PyTorch 1.6.0+cu101 Backend): 80.53%±0.48%
GAT(device=cuda:0, backend=PyTorch 1.6.0+cu101 Backend): 82.74%±0.58%
GCN(device=cuda:0, backend=PyTorch Geometric 1.6.1 (PyTorch 1.6.0+cu101) Backend): 81.22%±0.75%
SGC(device=cuda:0, backend=PyTorch Geometric 1.6.1 (PyTorch 1.6.0+cu101) Backend): 80.77%±0.58%
GAT(device=cuda:0, backend=PyTorch Geometric 1.6.1 (PyTorch 1.6.0+cu101) Backend): 82.72%±1.08%


Using backend: pytorch


GCN(device=cuda:0, backend=DGL PyTorch 0.5.2 (PyTorch 1.6.0+cu101) Backend): 81.39%±0.44%
SGC(device=cuda:0, backend=DGL PyTorch 0.5.2 (PyTorch 1.6.0+cu101) Backend): 80.70%±0.54%
GAT(device=cuda:0, backend=DGL PyTorch 0.5.2 (PyTorch 1.6.0+cu101) Backend): 82.18%±0.47%


# Results on Cora Dataset

In [5]:
import pandas as pd
table = pd.DataFrame(table, columns=GNNs)
table.index = Backends
table

,GCN,SGC,GAT
TensorFlow,80.48%±0.97%,81.33%±0.61%,81.96%±1.02%
PyTorch,81.25%±0.88%,80.53%±0.48%,82.74%±0.58%
PyG,81.22%±0.75%,80.77%±0.58%,82.72%±1.08%
DGL,81.39%±0.44%,80.70%±0.54%,82.18%±0.47%
